| **Chapter**                   | 6:  Modeling the control of infectious disease    |
| ----------------------------- |---------------------------------------------------|
| **Computational Note**        | 6.2: Waning immunity                              |
| **Package prerequisites**     | [JiTCDDE](https://github.com/neurophysik/jitcdde) |

# Waning immunity delay differential equation

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from jitcdde import jitcdde, y, t
from symengine import exp
import seaborn as sns
import pandas as pd

COLORS = {"S": "#2f4b7c",
          "E": "#ffa600",
          "I": "#f95d6a",
          "R": "#a05195",
          "D": "#003f5c",
          "C": "#ff7c43",
          "X": "#665191",
          "Y": "#d45087"}

In [ ]:
%load_ext watermark
%watermark --iversions -v -a "Chris von Csefalvay" -gu "chrisvoncsefalvay"

In [2]:
beta = 1.5
gamma = 1/14
mu = 0.0002
tau = 180
nu = 1/(100000 * 365)

In [3]:
f = [
    mu - beta * y(0, t) * y(1, t) - (nu + mu) * y(0, t) + nu * y(0, t - tau),
    beta * y(0, t) * y(1, t) - (gamma + mu) * y(1, t),
    nu * (y(0, t) - y(0, t-tau)) - mu * y(2, t),
    gamma * y(1, t) - mu * y(3, t)
]

In [4]:
DDE = jitcdde(f, n=4, delays=[tau])

In [5]:
DDE.constant_past([0.9, 0.1, 0, 0], time=tau)

In [6]:
DDE.step_on_discontinuities()

Generating, compiling, and loading C code.


In file included from /var/folders/1d/97x3559931s0_6hkz64q4zth0000gn/T/jitcxde_9rea753c/jitced.c:2:
In file included from /Users/csefalvayk/miniconda3/envs/jitcdde/include/python3.10/Python.h:25:
In file included from /usr/local/include/stdio.h:64:
/usr/local/include/_stdio.h:93:16: warning: pointer is missing a nullability type specifier (_Nonnull, _Nullable, or _Null_unspecified) [-Wnullability-completeness]
        unsigned char   *_base;
                        ^
/usr/local/include/_stdio.h:93:16: note: insert '_Nullable' if the pointer may be null
        unsigned char   *_base;
                        ^
                          _Nullable 
/usr/local/include/_stdio.h:93:16: note: insert '_Nonnull' if the pointer should never be null
        unsigned char   *_base;
                        ^
                          _Nonnull 
/usr/local/include/_stdio.h:138:32: warning: pointer is missing a nullability type specifier (_Nonnull, _Nullable, or _Null_unspecified) [-Wnullability-compl

In file included from /var/folders/1d/97x3559931s0_6hkz64q4zth0000gn/T/jitcxde_9rea753c/jitced.c:3:
In file included from /Users/csefalvayk/miniconda3/envs/jitcdde/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:6:
In file included from /Users/csefalvayk/miniconda3/envs/jitcdde/lib/python3.10/site-packages/numpy/core/include/numpy/npy_interrupt.h:23:
/usr/local/include/signal.h:69:21: warning: pointer is missing a nullability type specifier (_Nonnull, _Nullable, or _Null_unspecified) [-Wnullability-completeness]
extern __const char *__const sys_signame[NSIG];
                    ^
/usr/local/include/signal.h:69:21: note: insert '_Nullable' if the pointer may be null
extern __const char *__const sys_signame[NSIG];
                    ^
                      _Nullable 
/usr/local/include/signal.h:69:21: note: insert '_Nonnull' if the pointer should never be null
extern __const char *__const sys_signame[NSIG];
                    ^
                      _Nonnull 
/usr/

219 warnings generated.


Using default integration parameters.


array([ 2.46061666e-02,  3.41792289e-05, -4.27167918e-06,  9.75363926e-01])

In [7]:
res = []

for time in np.arange(DDE.t, DDE.t + 30000, 1):
    res.append(DDE.integrate(time))

/Users/csefalvayk/miniconda3/envs/jitcdde/lib/python3.10/site-packages/jitcdde/_jitcdde.py:792: UserWarning: The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.
  warn("The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.")


In [8]:
np.vstack(res)[3:,:]

array([[ 2.51876306e-02,  3.08386336e-05, -4.30864047e-06,
         9.74785839e-01],
       [ 2.53814247e-02,  2.98166326e-05, -4.30892327e-06,
         9.74593068e-01],
       [ 2.55752083e-02,  2.88368807e-05, -4.30791709e-06,
         9.74400263e-01],
       ...,
       [ 4.77523810e-02,  2.65884856e-03, -1.02334744e-08,
         9.49588781e-01],
       [ 4.77523810e-02,  2.65884856e-03, -1.02314279e-08,
         9.49588781e-01],
       [ 4.77523810e-02,  2.65884856e-03, -1.02293819e-08,
         9.49588781e-01]])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, facecolor="w", figsize=(6, 9), dpi=600)

ax1.plot(np.vstack(res)[:,0], color=COLORS["S"], alpha=0.5, lw=2, label="Susceptible")
ax1.plot(np.vstack(res)[:,1], color=COLORS["I"], alpha=0.5, lw=2, label="Infectious")

ax1.set_xlabel('Time (days)')
ax1.set_ylabel("Fraction of population in compartment")
ax1.grid(b=True, which='major', c='w', lw=2, ls='-')

for spine in ("top", "right"):
    ax1.spines[spine].set_visible(False)
    ax2.spines[spine].set_visible(False)

ax1.set_xlim(365, 3000)
ax1.set_ylim(0.0001, 0.1)

legend = ax1.legend()
legend.get_frame().set_alpha(0.5)

fig.subplots_adjust(bottom=0.25)
ax1.legend(title='', bbox_to_anchor=(0.5, -0.3), loc="lower center", ncol=3, frameon=False)

ax1.set_yscale("log")

S, I = np.vstack(res)[365:, 0], np.vstack(res)[365:, 1]

ax2.plot(S, I, c=COLORS["X"], lw=1)
ax2.set_xlabel("$S$")
ax2.set_ylabel("$I$")

ax2.axhline(y=I[-1], color=COLORS["I"], linestyle=":", alpha=0.5)
ax2.axvline(x=S[-1], color=COLORS["S"], linestyle=":", alpha=0.5)

ax2.scatter(np.vstack(res)[-1, 0], np.vstack(res)[-1, 1], color=COLORS["I"], alpha=0.825)
ax2.annotate(xy=[np.vstack(res)[-1, 0] + 0.0017, np.stack(res)[-1, 1] + 0.0004], text="EE")

fig.tight_layout()
plt.subplots_adjust(hspace=0.4)

for i in range(10, 600, 50):
    if np.sqrt((S[i + 1] - S[i])**2 + (I[i + 1] - I[i]) ** 2) > 0.0001:
        ax2.arrow(x=S[i + 1], y=I[i + 1],
                  dx=S[i + 1] - S[i], dy=I[i + 1] - I[i],
                  shape="full",
                  lw=0,
                  length_includes_head=True,
                  head_starts_at_zero=True,
                  head_width=.00055,
                  width=0.0002,
                  color=COLORS["X"])


plt.savefig("vaccination_waning_integral.pdf")
plt.show()